In [20]:
##########################################
#Examples for different configurations:
#
# Assume example data set:
# - Patient1
#   - pat1_TP1_MR
#   - pat1_TP2_MR
#   - pat1_TP1_CT
# - Patient2
#   - pat2_TP1_MR1
#   - pat2_TP1_MR2
#   - pat2_TP2_MR1
#   - pat2_TP1_CT
#   - pat2_TP2_CT

In [21]:
###############################################################################
# Imports
###############################################################################
import os

import avid.common.workflow as workflow
import avid.common.artefact.defaultProps as artefactProps

from avid.selectors import ActionTagSelector as ATS
from avid.actions.pythonAction import PythonNaryBatchAction as naryPythonAction, PythonNaryBatchActionV2

from avid.linkers import CaseLinker, TimePointLinker, FractionLinker
from avid.splitter import SingleSplitter, CaseSplitter, KeyValueSplitter

In [22]:
###############################################################################
# Define ActionTagSelectors
###############################################################################
mr_image_selector = ATS('MR')
ct_image_selector = ATS('CT')
mask_selector = ATS('Seg')

In [23]:
###############################################################################
# Define callable
###############################################################################
def my_function(outputs, **kwargs):
    '''
        Simple print-callable that outputs the combinations of input artifacts.
    '''
    keys = kwargs.keys()
    print_output = 'output: {'
    
    for i, key in enumerate(keys):
        if i != 0:
            print_output += ', '
        print_output += '{}:{}'.format(key, [os.path.basename(mr) for mr in kwargs[key]])
    print_output += '}'
    
    print(print_output)
    with open(outputs[0], "w") as ofile:
        ofile.write(str(kwargs))

In [24]:
###############################################################################
# Initialize session
###############################################################################
session =  workflow.initSession(bootstrapArtefacts=os.path.join(os.getcwd(),'output', 'example.avid'),
                                sessionPath=os.path.join(os.getcwd(),'output', 'example'),
                                expandPaths=True,
                                debug=True)

In [25]:
###############################################################################
# Example 1: only select a specific modality (here MR) and make a call per image
###############################################################################
with session:
    PythonNaryBatchActionV2(primaryInputSelector=mr_image_selector, 
                        actionTag="example_1", 
                        generateCallable=my_function,
                        passOnlyURLs=True).do()

2021-12-14 11:35:42,042 [INFO] Starting action: PythonNaryBatchActionV2_example_1 (UID: 1f68e2af-76df-4caf-8281-73f0fa70c466) ...
2021-12-14 11:35:42,044 [INFO] Starting action: my_function (UID: fa83287b-6cdd-4952-9566-05579e1c0f89) ...
output: {primaryInput:['pat1_TP1_MR.txt']}
2021-12-14 11:35:42,066 [INFO] Finished action: my_function (UID: fa83287b-6cdd-4952-9566-05579e1c0f89) -> SUCCESS
2021-12-14 11:35:42,067 [INFO] Starting action: my_function (UID: e3a95334-9b6e-418a-b66e-65c6a8ece5d5) ...
output: {primaryInput:['pat1_TP2_MR.txt']}
2021-12-14 11:35:42,072 [INFO] Finished action: my_function (UID: e3a95334-9b6e-418a-b66e-65c6a8ece5d5) -> SUCCESS
2021-12-14 11:35:42,073 [INFO] Starting action: my_function (UID: dd2eedd3-3b51-4bd4-a9f0-9e677680d270) ...
output: {primaryInput:['pat2_TP1_MR1.txt']}
2021-12-14 11:35:42,079 [INFO] Finished action: my_function (UID: dd2eedd3-3b51-4bd4-a9f0-9e677680d270) -> SUCCESS
2021-12-14 11:35:42,081 [INFO] Starting action: my_function (UID: 5d5ce

In [26]:
###############################################################################
#Example 2: like example 1 but choose another variable name
###############################################################################
with session:
    PythonNaryBatchActionV2(primaryInputSelector=mr_image_selector,
                        primaryAlias='mr_images',
                        actionTag="example_2", 
                        generateCallable=my_function, 
                        passOnlyURLs=True).do()

2021-12-14 11:35:42,237 [INFO] Starting action: PythonNaryBatchActionV2_example_2 (UID: 5615130f-50a4-4b18-9523-4579b83600ee) ...
2021-12-14 11:35:42,239 [INFO] Starting action: my_function (UID: 37f211f0-cf0a-40d2-9285-1625dd113384) ...
output: {mr_images:['pat1_TP1_MR.txt']}
2021-12-14 11:35:42,249 [INFO] Finished action: my_function (UID: 37f211f0-cf0a-40d2-9285-1625dd113384) -> SUCCESS
2021-12-14 11:35:42,250 [INFO] Starting action: my_function (UID: 6830282b-f1bf-43f3-bad1-264aa8449319) ...
output: {mr_images:['pat1_TP2_MR.txt']}
2021-12-14 11:35:42,252 [INFO] Finished action: my_function (UID: 6830282b-f1bf-43f3-bad1-264aa8449319) -> SUCCESS
2021-12-14 11:35:42,253 [INFO] Starting action: my_function (UID: 84d4e713-8f25-44ff-8d48-69c1528dbf9f) ...
output: {mr_images:['pat2_TP1_MR1.txt']}
2021-12-14 11:35:42,261 [INFO] Finished action: my_function (UID: 84d4e713-8f25-44ff-8d48-69c1528dbf9f) -> SUCCESS
2021-12-14 11:35:42,262 [INFO] Starting action: my_function (UID: 7f53a5c1-3caa-

In [27]:
###############################################################################
#Example 3: select MR and CT images and call them pairwise (folding both sets per case)
###############################################################################
with session:
    PythonNaryBatchActionV2(primaryInputSelector=mr_image_selector,
                        primaryAlias='mr_images',
                        additionalInputSelectors = {'ct_images': ct_image_selector},
                        actionTag="example_3",
                        generateCallable=my_function,
                        passOnlyURLs=True).do().tagSelector

2021-12-14 11:35:42,427 [INFO] Starting action: PythonNaryBatchActionV2_example_3 (UID: eb332fdd-1083-4fe7-bbd0-c2f4a41e4a53) ...
2021-12-14 11:35:42,430 [INFO] Starting action: my_function (UID: dba55d6e-f199-4059-8b8a-436cdf134e03) ...
output: {mr_images:['pat1_TP1_MR.txt'], ct_images:['pat1_TP1_CT.txt']}
2021-12-14 11:35:42,455 [INFO] Finished action: my_function (UID: dba55d6e-f199-4059-8b8a-436cdf134e03) -> SUCCESS
2021-12-14 11:35:42,456 [INFO] Starting action: my_function (UID: 439f4c14-0268-4312-b870-70519f65bb83) ...
output: {mr_images:['pat1_TP2_MR.txt'], ct_images:['pat1_TP1_CT.txt']}
2021-12-14 11:35:42,458 [INFO] Finished action: my_function (UID: 439f4c14-0268-4312-b870-70519f65bb83) -> SUCCESS
2021-12-14 11:35:42,459 [INFO] Starting action: my_function (UID: 212da937-7535-4b79-9b10-5ad4b48fc29a) ...
output: {mr_images:['pat2_TP1_MR1.txt'], ct_images:['pat2_TP1_CT.txt']}
2021-12-14 11:35:42,463 [INFO] Finished action: my_function (UID: 212da937-7535-4b79-9b10-5ad4b48fc29a

In [28]:
###############################################################################
#Example 3b:  select MR and CT images and call them pairwise (but only MRs and CTs of same patient and timepoint)
###############################################################################
with session:
    splitProperties=[artefactProps.CASE]
    PythonNaryBatchActionV2(primaryInputSelector=mr_image_selector,
                            primaryAlias='mr_images',
                            additionalInputSelectors = {'additional': ct_image_selector},
                            linker={'additional': CaseLinker()+TimePointLinker()},
                            actionTag="example_3b",
                            generateCallable=my_function,
                            passOnlyURLs=True).do().tagSelector

2021-12-14 11:35:42,963 [INFO] Starting action: PythonNaryBatchActionV2_example_3b (UID: 03c58b69-bbc9-4f58-a97e-de3edc0da99d) ...
2021-12-14 11:35:42,966 [INFO] Starting action: my_function (UID: 27e0b15e-805e-4ac1-aa8b-b5d5cbddd0cc) ...
output: {mr_images:['pat1_TP1_MR.txt'], additional:['pat1_TP1_CT.txt']}
2021-12-14 11:35:42,985 [INFO] Finished action: my_function (UID: 27e0b15e-805e-4ac1-aa8b-b5d5cbddd0cc) -> SUCCESS
2021-12-14 11:35:42,986 [INFO] Starting action: my_function (UID: 72ebd0e6-5d5c-4fa3-b44b-7fecae8e14a4) ...
output: {mr_images:['pat2_TP1_MR1.txt'], additional:['pat2_TP1_CT.txt']}
2021-12-14 11:35:42,991 [INFO] Finished action: my_function (UID: 72ebd0e6-5d5c-4fa3-b44b-7fecae8e14a4) -> SUCCESS
2021-12-14 11:35:42,992 [INFO] Starting action: my_function (UID: 9ecee410-5c29-4e51-9b90-3e540667db9a) ...
output: {mr_images:['pat2_TP1_MR2.txt'], additional:['pat2_TP1_CT.txt']}
2021-12-14 11:35:42,994 [INFO] Finished action: my_function (UID: 9ecee410-5c29-4e51-9b90-3e54066

In [29]:
###############################################################################
#Example 3c:  select MR and CT images and call them blockwise (but only MRs and CTs of same patient)
###############################################################################
with session:
    splitProperties=[artefactProps.CASE]
    PythonNaryBatchActionV2(primaryInputSelector=mr_image_selector,
                        primaryAlias='mr_images',
                        additionalInputSelectors = {'additional': ct_image_selector},
                        linker={'additional': CaseLinker(allowOnlyFullLinkage=False)+TimePointLinker(allowOnlyFullLinkage=False)},
                        splitter={'mr_images': KeyValueSplitter(*splitProperties), 'additional': CaseSplitter()},
                        actionTag="example_3b", 
                        generateCallable=my_function,
                        passOnlyURLs=True).do().tagSelector

2021-12-14 11:35:43,556 [INFO] Starting action: PythonNaryBatchActionV2_example_3b (UID: f7f266a6-de27-4995-a791-68bbea2509c2) ...
2021-12-14 11:35:43,559 [INFO] Starting action: my_function (UID: ecbc8858-c972-4989-988d-2bd7d2d13560) ...
output: {mr_images:['pat1_TP1_MR.txt', 'pat1_TP2_MR.txt'], additional:['pat1_TP1_CT.txt']}
2021-12-14 11:35:43,562 [INFO] Finished action: my_function (UID: ecbc8858-c972-4989-988d-2bd7d2d13560) -> SUCCESS
2021-12-14 11:35:43,563 [INFO] Starting action: my_function (UID: 6b59ac53-a111-4701-985e-f02128016c57) ...
output: {mr_images:['pat2_TP1_MR1.txt', 'pat2_TP1_MR2.txt', 'pat2_TP2_MR1.txt'], additional:['pat2_TP1_CT.txt', 'pat2_TP2_CT.txt']}
2021-12-14 11:35:43,566 [INFO] Finished action: my_function (UID: 6b59ac53-a111-4701-985e-f02128016c57) -> SUCCESS
2021-12-14 11:35:43,567 [INFO] Finished action: PythonNaryBatchActionV2_example_3b (UID: f7f266a6-de27-4995-a791-68bbea2509c2) -> SUCCESS
2021-12-14 11:35:43,568 [INFO] Successful actions: 4.
2021-12-

In [32]:
###############################################################################
#Example 4: select MR images and masks and call them block wise per case
###############################################################################
with session:
    PythonNaryBatchActionV2(primaryInputSelector=mr_image_selector,
                        primaryAlias='mr_images',
                        additionalInputSelectors ={'additional': mask_selector},
                        linker={'additional': CaseLinker(allowOnlyFullLinkage=False)+TimePointLinker(allowOnlyFullLinkage=False)},
                        splitter={'mr_images': CaseSplitter(), 'additional': CaseSplitter()},
                        actionTag="example_4", 
                        generateCallable=my_function, 
                        passOnlyURLs=True).do().tagSelector

2021-12-14 11:36:00,586 [INFO] Starting action: PythonNaryBatchActionV2_example_4 (UID: e6a9806c-41ed-48e7-81a5-b7502dddd5b4) ...
2021-12-14 11:36:00,587 [INFO] Starting action: my_function (UID: ca34eb3f-bbdd-47ee-bd37-eeb99005f4ce) ...
output: {mr_images:['pat1_TP1_MR.txt', 'pat1_TP2_MR.txt'], additional:['pat1_TP1_Seg1.txt', 'pat1_TP1_Seg2.txt', 'pat1_TP2_Seg1.txt']}
2021-12-14 11:36:00,591 [INFO] Finished action: my_function (UID: ca34eb3f-bbdd-47ee-bd37-eeb99005f4ce) -> SUCCESS
2021-12-14 11:36:00,592 [INFO] Starting action: my_function (UID: 77ca6407-8bd0-41c4-a1f2-4f0a54f6f48c) ...
output: {mr_images:['pat2_TP1_MR1.txt', 'pat2_TP1_MR2.txt', 'pat2_TP2_MR1.txt'], additional:['pat2_TP1_Seg1.txt', 'pat2_TP2_Seg1.txt']}
2021-12-14 11:36:00,595 [INFO] Finished action: my_function (UID: 77ca6407-8bd0-41c4-a1f2-4f0a54f6f48c) -> SUCCESS
2021-12-14 11:36:00,595 [INFO] Starting action: my_function (UID: a81bd569-4c4d-4db7-b807-52b344715427) ...
output: {mr_images:['pat3_TP1_MR.txt'], addit